# 4. Training and evaluating the model

## 4.1. Training

### 4.1.1. Parent model

After the previous step we ended up with folder "prepared_ru_parent" with prepared data for the model. Our next step will be training the model. I used 1 GPU so this was the training command I ran:
```bash
sockeye-train \
    --output model_ru_parent \
    --decode-and-evaluate -1 \
    --checkpoint-interval 4000 \
    --gradient-clipping-type abs \
    --initial-learning-rate 0.0001 \
    --keep-last-params 2 \
    --batch-size 8192 \
    --max-seq-len 200 \
    --num-layers 6 \
    --weight-tying none \
    --max-num-epochs 10000 \
    --optimized-metric bleu \
    --prepared-data prepared_ru_parent \
    --validation-source=val_parent.ru.BPE \
    --validation-target=val_parent.chv.BPE
```

### 4.1.2. Child model

As a result of this command we get a "model_ru_parent" folder with the weights. This folder (in particular, best weights from it) we mention in a training command for child model. I trained child model on 1 GPU so this was the command:
```bash
sockeye-train \
    --output model_ru_child \
    --params model_ru_parent/params.best \
    --decode-and-evaluate -1 \
    --checkpoint-interval 4000 \
    --gradient-clipping-type abs \
    --initial-learning-rate 0.0001 \
    --keep-last-params 2 \
    --batch-size 8192 \
    --max-seq-len 200 \
    --num-layers 6 \
    --weight-tying none \
    --max-num-epochs 10000 \
    --optimized-metric bleu \
    --prepared-data prepared_ru_child \
    --validation-source=val_child.ru.BPE \
    --validation-target=val_child.kh.BPE
```

## 4.2. Evaluation

In order to evaluate the model we need to translate the sentences from the test set (those that model didn't see during training) and then compare them to the reference translation. This is command for translating:
```bash
sockeye-translate --input test_child.ru.BPE --output out.bpe --model model_ru_child --dtype float32 --beam-size 5 --batch-size 64
```

After that we get bask from tokenized lines to actual texts with this command:
```bash
sed -E 's/(@@ |@@$)//g' <out.bpe >out.tok
```

After that we use sacrebleu tool to check BLEU metric:
```bash
sacrebleu test_child.kh -tok none -i out.tok
```

And ChrF metric:
```bash
sacrebleu test_child.kh --metrics chrf -i out.tok
```